<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-17 14:27:53
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.39 C
-------------------
Today PnL: 27.91 K (0.2%)
Current PnL: -12.62 L (-8.79%)
CY Booked + Current PnL: -4.54 L (-3.17%)
-------------------
Total profit:  4.21 L
Total loss:  -16.84 L
-------------------
Total Booked + Current PnL: 22.44 L (19.25%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.83%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 70.91 L (51.18%)
Deployed:  1.17 C
Current:  1.39 C
CAGR/XIRR %: 11.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-228.35,H-SC,9.65,92586.0,-2730.0,2768.0,-1.12,-2.86,2.99,0.04,162.0,-0.99,0.67,82.62,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-8.66,M-LC,4.20,220493.0,19014.0,8202.0,0.04,9.44,3.72,13.51,61.0,2.32,1.59,23.22,XY25,NTT,BANKS
86,UJJIVANSFB,52.77,53.00,52.86,M-SC,24.72,132489.0,-9990.0,10612.0,-1.03,-7.01,8.01,0.44,251.0,-0.94,0.96,56.37,OX40N,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,16.86,H-LC,9.54,104868.0,11435.0,11158.0,0.68,12.24,10.64,24.18,31.0,1.02,0.76,27.34,XY25,ATH,FMCG
4,APOLLOHOSP,6904.43,8285.00,-18.95,H-LC,5.83,110482.0,6916.0,13788.0,0.18,6.68,12.48,20.00,14.0,0.50,0.80,21.69,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.00,-19.88,M-SC,1.16,166888.0,-887.0,56725.0,1.82,-0.53,33.99,33.28,210.0,-0.02,1.21,29.21,XY24,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.77,H-SC,20.39,100319.0,-644.0,95173.0,1.02,-0.64,94.87,93.63,150.0,-0.01,0.72,49.42,SR,ATH,CHEMICALS
12,BANKINDIA,116.91,190.00,-32.52,H-MC,10.02,178746.0,-1062.0,113468.0,-1.53,-0.59,63.48,62.52,93.0,-0.01,1.29,33.30,XR,NTT,BANKS
61,PGHH,13388.00,18062.58,-20.22,H-MC,7.85,201375.0,555.0,69555.0,1.67,0.28,34.54,34.92,92.0,0.01,1.45,4.97,X40,ATH,FMCG
59,NATIONALUM,189.63,247.44,-50.15,H-MC,5.25,100165.0,-339.0,30981.0,0.52,-0.34,30.93,30.49,83.0,-0.01,0.72,31.96,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-22.84,H-LC,0.27,203198.0,-1302.0,42712.0,0.50,-0.64,21.02,20.25,10.0,-0.03,1.47,7.11,X40N,BTT,PHARMA
40,HINDUNILVR,2413.81,2723.00,-9.90,H-LC,1.10,282374.0,12027.0,22590.0,0.14,4.45,8.00,12.81,16.0,0.53,2.04,18.09,XY25,NTT,FMCG
37,HCLTECH,1647.60,1972.35,6.70,H-LC,2.37,94471.0,-6033.0,25847.0,-0.90,-6.00,27.36,19.71,12.0,-0.23,0.68,14.01,X40,ATH,IT
10,BAJAJHFL,122.26,152.00,-2.81,H-LC,2.96,201589.0,1083.0,47696.0,-0.36,0.54,23.66,24.33,29.0,0.02,1.46,16.25,X40N,BTT,FINANCE
41,HINDZINC,471.44,730.22,16.43,H-LC,3.35,190073.0,-15003.0,127577.0,0.40,-7.32,67.12,54.89,2.0,-0.12,1.37,15.02,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,SONACOMS,578.05,1006.00,-33.17,M-SC,14.98,86100.0,-15059.0,89949.0,8.12,-14.89,104.47,74.03,202.0,-0.17,0.62,21.41,AR,BTT,AUTO
8,AWL,325.39,485.00,-67.06,H-MC,7.97,228649.0,-39798.0,171487.0,5.64,-14.83,75.00,49.05,82.0,-0.23,1.65,17.56,XY24,NTT,FMCG
64,RELAXO,902.64,1176.00,-27.28,H-SC,18.50,65773.0,-49765.0,84755.0,3.85,-43.07,128.86,30.28,131.0,-0.59,0.48,28.49,X40N,NTT,FOOTWEAR
91,VBL,492.64,672.28,-12.87,H-LC,14.95,264642.0,-398.0,97044.0,2.46,-0.15,36.67,36.46,3.0,-0.00,1.91,13.02,X40N,ATH,FMCG
94,WHIRLPOOL,1219.98,2270.00,-37.29,M-SC,12.13,105578.0,14080.0,64677.0,2.44,15.39,61.26,86.07,205.0,0.22,0.76,52.73,XR,NTT,DURABLES


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19495.0,-30098.0,1298.0,-6.25,-60.69,6.66,-58.07,201.0,-23.19,0.14,30.74,XY24,ATH,MISC
56,LTIM,5564.16,7230.20,-2.47,H-LC,3.71,187650.0,-12660.0,72639.0,-2.15,-6.32,38.71,29.94,28.0,-0.17,1.36,29.64,X200,ATH,IT
80,TATAELXSI,7332.28,7332.00,-17.31,H-MC,0.77,86884.0,-15768.0,15761.0,-1.66,-15.36,18.14,-0.00,54.0,-1.00,0.63,32.07,OX40N,NTT,IT
47,INDUSINDBK,1354.50,1800.00,82.01,L-MC,12.20,49365.0,-27842.0,53235.0,-1.56,-36.06,107.84,32.89,260.0,-0.52,0.36,35.95,XR,NTT,BANKS
87,UNIONBANK,123.87,163.00,-4.47,M-LC,18.54,166025.0,25185.0,19309.0,-1.56,17.88,11.63,31.59,73.0,1.30,1.20,44.22,XY24,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-228.35,H-SC,9.65,92586.0,-2730.0,2768.0,-1.12,-2.86,2.99,0.04,162.0,-0.99,0.67,82.62,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,52.86,M-SC,24.72,132489.0,-9990.0,10612.0,-1.03,-7.01,8.01,0.44,251.0,-0.94,0.96,56.37,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-27.47,M-SC,12.80,154528.0,-16613.0,16565.0,-0.55,-9.71,10.72,-0.03,195.0,-1.00,1.12,11.06,OX40N,NTT,DURABLES
21,CERA,7989.07,8422.0,-10.74,H-SC,2.00,127494.0,-16309.0,24096.0,2.27,-11.34,18.90,5.42,155.0,-0.68,0.92,40.69,OX40N,NTT,CERAMICS
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,10.22,149048.0,-25511.0,25547.0,0.14,-14.61,17.14,0.02,129.0,-1.00,1.08,29.25,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77560.0,43111.0,1854.0,0.52,125.15,2.39,130.54,1.0,23.25,0.56,18.60,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,16.43,H-LC,3.35,190073.0,-15003.0,127577.0,0.40,-7.32,67.12,54.89,2.0,-0.12,1.37,15.02,X5K,ATH,METALS
91,VBL,492.64,672.28,-12.87,H-LC,14.95,264642.0,-398.0,97044.0,2.46,-0.15,36.67,36.46,3.0,-0.00,1.91,13.02,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-22.88,H-LC,10.73,251074.0,-44860.0,138769.0,-0.44,-15.16,55.27,31.73,4.0,-0.32,1.81,0.84,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,H-LC,7.46,284646.0,23045.0,122569.0,-1.10,8.81,43.06,55.67,6.0,0.19,2.06,15.45,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,249.44,NaN,NaN,0.90,54568.0,13313.0,60.0,-0.11,32.27,0.11,32.41,196.0,221.88,0.39,28.47,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,21.8,H-SC,6.15,156159.0,29959.0,16209.0,0.58,23.74,10.38,36.59,153.0,1.85,1.13,39.23,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77560.0,43111.0,1854.0,0.52,125.15,2.39,130.54,1.0,23.25,0.56,18.60,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,91.93,M-MC,11.77,229887.0,4925.0,162737.0,-0.06,2.19,70.79,74.53,194.0,0.03,1.66,33.47,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,35.49,H-SC,9.78,138713.0,13416.0,45664.0,-1.08,10.71,32.92,47.15,141.0,0.29,1.00,73.54,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,21.80,H-SC,6.15,156159.0,29959.0,16209.0,0.58,23.74,10.38,36.59,153.0,1.85,1.13,39.23,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,5.43,H-SC,3.16,118457.0,16453.0,27897.0,0.56,16.13,23.55,43.48,122.0,0.59,0.86,35.61,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,0.25,H-SC,12.40,188120.0,23450.0,45638.0,1.11,14.24,24.26,41.95,119.0,0.51,1.36,22.77,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,0.34,77560.0,43111.0,1854.0,0.52,125.15,2.39,130.54,1.0,23.25,0.56,18.60,AR,ATH,ELECTRICAL
53,KPIGREEN,497.21,731.64,35.49,H-SC,9.78,138713.0,13416.0,45664.0,-1.08,10.71,32.92,47.15,141.0,0.29,1.00,73.54,MH,ATH,POWER
57,MASFIN,326.60,399.50,-9.67,H-SC,13.98,102735.0,4755.0,17116.0,-1.08,4.85,16.66,22.32,146.0,0.28,0.74,49.00,XR,ATH,FINANCE
94,WHIRLPOOL,1219.98,2270.00,-37.29,M-SC,12.13,105578.0,14080.0,64677.0,2.44,15.39,61.26,86.07,205.0,0.22,0.76,52.73,XR,NTT,DURABLES
45,INDIAMART,2327.09,4911.36,-46.58,H-SC,10.91,139236.0,15900.0,121066.0,-1.20,12.89,86.95,111.05,118.0,0.13,1.01,36.43,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19495.0,-30098.0,1298.0,-6.25,-60.69,6.66,-58.07,201.0,-23.19,0.14,30.74,XY24,ATH,MISC
28,DIGITIDE,188.38,249.44,NaN,NaN,0.90,54568.0,13313.0,60.0,-0.11,32.27,0.11,32.41,196.0,221.88,0.39,28.47,XY24,ATH,IT
27,DEN,52.79,75.00,133.16,L-SC,6.18,65584.0,-18880.0,54415.0,0.07,-22.35,82.97,42.07,266.0,-0.35,0.47,39.28,AR,NTT,ENTERTAINMENT
74,SIS,477.00,477.00,2314.51,M-SC,4.34,64957.0,-16133.0,16135.0,-0.22,-19.90,24.84,0.00,237.0,-1.00,0.47,31.65,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-47.69,L-SC,20.22,77722.0,-35827.0,75880.0,1.30,-31.55,97.63,35.27,268.0,-0.47,0.56,90.23,XR,NTT,HOTELS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.69,19495.0,-30098.0,1298.0,-6.25,-60.69,6.66,-58.07,201.0,-23.19,0.14,30.74,XY24,ATH,MISC
64,RELAXO,902.64,1176.00,-27.28,H-SC,18.50,65773.0,-49765.0,84755.0,3.85,-43.07,128.86,30.28,131.0,-0.59,0.48,28.49,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,7.77,H-SC,20.39,100319.0,-644.0,95173.0,1.02,-0.64,94.87,93.63,150.0,-0.01,0.72,49.42,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-47.69,L-SC,20.22,77722.0,-35827.0,75880.0,1.30,-31.55,97.63,35.27,268.0,-0.47,0.56,90.23,XR,NTT,HOTELS
57,MASFIN,326.60,399.50,-9.67,H-SC,13.98,102735.0,4755.0,17116.0,-1.08,4.85,16.66,22.32,146.0,0.28,0.74,49.00,XR,ATH,FINANCE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.40
1,25,40.81
2,50,69.86


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,42.09
LC,34.81
MC,21.99


In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,29.74
X40N,14.37
X40,12.46
XY25,10.93
XR,9.36
OX40N,8.53
AR,8.16
X5K,2.15
MH,1.72


In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-LC,28.03
H-SC,25.66
H-MC,18.93
M-SC,14.92
M-LC,5.75
M-MC,2.70
L-SC,1.51
L-LC,1.03
L-MC,0.36


In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.17,0.26,33.11
IT,11.77,-11.45,66.39
FINANCE,8.52,-10.54,50.66
BANKS,7.75,-6.24,49.59
MISC,5.95,-16.77,71.70
PAINTS,5.55,-13.63,36.02
ELECTRICAL,5.31,-0.30,47.83
HEALTHCARE,4.37,0.24,26.25
AUTO,4.34,-12.62,62.14


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2592063.0,25
AR,1022462.0,10
XR,997020.0,13
X40,729324.0,10
X40N,564175.0,12
XY25,424819.0,8
OX40N,294114.0,11
SR,176733.0,2
X5K,141366.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2273835.0,26
M-SC,1565372.0,20
H-LC,1305248.0,21
H-MC,1086857.0,15
M-MC,303345.0,2
M-LC,264856.0,4
L-SC,189092.0,3
L-MC,53235.0,1
L-LC,46308.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,885438.0,7
M-SC,XY24,772386.0,7
H-SC,AR,518081.0,3
H-LC,X40,471280.0,6
H-MC,XY24,423713.0,4
H-SC,XR,327964.0,4
M-MC,XY24,303345.0,2
H-LC,X40N,291609.0,6
M-SC,XR,269349.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
